In [2]:
import pandas as pd
import numpy as np
import quandl
import math
from sklearn import preprocessing, cross_validation, svm
from sklearn.linear_model import LinearRegression

/home/cal/.local/lib/python3.5/site-packages/pandas/core/computation/__init__.py:18: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
df = quandl.get('WIKI/AMD')
df.head()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
1983-03-21,35.88,36.13,35.25,36.00,127700.0,0.0,1.0,8.967543,9.030026,8.810086,8.997535,510800.0
1983-03-22,34.88,35.88,34.00,34.00,82000.0,0.0,1.0,8.717612,8.967543,8.497672,8.497672,328000.0
1983-03-23,34.00,35.25,33.88,34.88,106800.0,0.0,1.0,8.497672,8.810086,8.467680,8.717612,427200.0
1983-03-24,34.88,35.13,34.63,35.13,98300.0,0.0,1.0,8.717612,8.780094,8.655129,8.780094,393200.0
1983-03-25,35.63,36.25,35.00,35.50,52600.0,0.0,1.0,8.905060,9.060018,8.747603,8.872569,210400.0


In [4]:
df.shape

(8704, 12)

In [5]:
df = df[['Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume']]
df['HL_pct'] = (df['Adj. High'] - df['Adj. Close']) / df['Adj. Close'] * 100
df['pct_change'] = (df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open'] * 100
df.head()

,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume,HL_pct,pct_change
Date,,,,,,,
1983-03-21,8.967543,9.030026,8.810086,8.997535,510800.0,0.361111,0.334448
1983-03-22,8.717612,8.967543,8.497672,8.497672,328000.0,5.529412,-2.522936
1983-03-23,8.497672,8.810086,8.467680,8.717612,427200.0,1.060780,2.588235
1983-03-24,8.717612,8.780094,8.655129,8.780094,393200.0,0.000000,0.716743
1983-03-25,8.905060,9.060018,8.747603,8.872569,210400.0,2.112676,-0.364861


In [6]:
df = df[['Adj. Close', 'HL_pct', 'pct_change', 'Adj. Volume']]
df.tail()

,Adj. Close,HL_pct,pct_change,Adj. Volume
Date,,,,
2017-09-19,13.12,1.295732,-0.981132,64693928.0
2017-09-20,13.76,0.290698,5.279265,82385985.0
2017-09-21,13.41,6.189411,-4.214286,164802839.0
2017-09-22,13.30,0.751880,0.757576,49752570.0
2017-09-25,12.61,5.313243,-4.830189,83429167.0


In [7]:
forecast_col = 'Adj. Close'
df.fillna('-99999', inplace=True)
forecast_out = int(math.ceil(0.01*len(df)))
df['label'] = df[forecast_col].shift(-forecast_out)
df.dropna(inplace=True)
df.head()

,Adj. Close,HL_pct,pct_change,Adj. Volume,label
Date,,,,,
1983-03-21,8.997535,0.361111,0.334448,510800.0,16.557964
1983-03-22,8.497672,5.529412,-2.522936,328000.0,15.965626
1983-03-23,8.717612,1.060780,2.588235,427200.0,14.840934
1983-03-24,8.780094,0.000000,0.716743,393200.0,14.870926
1983-03-25,8.872569,2.112676,-0.364861,210400.0,14.995892


In [8]:
df.tail()

,Adj. Close,HL_pct,pct_change,Adj. Volume,label
Date,,,,,
2017-05-15,11.42,1.576182,0.087642,52210042.0,13.12
2017-05-16,12.75,0.156863,9.348199,155882385.0,13.76
2017-05-17,11.20,9.691964,-5.165114,158929483.0,13.41
2017-05-18,11.28,1.861702,2.638763,85319480.0,13.30
2017-05-19,11.40,1.929825,-0.610288,49828043.0,12.61


In [15]:
x = np.array(df.drop(['label'], 1))
y = np.array(df['label'])
x = preprocessing.scale(x)
#x = x[:-forcast_out+1]
#df.dropna(inplace=True)
y = np.array(df['label'])
x_train, x_test, y_train, y_test = cross_validation.train_test_split(x, y, test_size=0.2)
#clf = LinearRegression()  #0.559298101954
clf = svm.SVR() #.58699059901
clf.fit(x_train, y_train)
confidence = clf.score(x_test, y_test)
print(confidence)

0.559298101954


In [12]:
print(confidence)

0.58699059901


In [ ]:
machine learning 4